# ***Readme***

ΣΤΙΒΑΚΤΑΣ ΕΜΜΑΝΟΥΗΛ
Α.Μ = 1115201700152


Έχουν υλοποιηθεί όλα τα ερωτήματα της εργασίας . Μερικές παρατηρήσεις:


1) Έχει χρησιμοποιηθεί κώδικας απο την προηγούμενη εργασία όπως η συνάρτηση find_classification(). Όπου χρησιμοποιήθηκαν πηγές από το διαδίκτυο αναγράφονται στα σχόλια.


2) Διαβάζω τα train και test δεδομενα. Έπειτα από το κατάλληλο text cleaning εκπαιδέυω τα δεδομένα με τους αλγορίθμους ταξινόμησης .


3) Για το count vectorizer τα καλύτερα ποσοστά βγήκαν στον naive bayes όταν χρησιμοποιήθηκε laplace smoothing.  Bigrams και lemmatization αύξησαν εξίσου τα ποσοστά, όχι όμως όπως με τον laplace. Να τονιστεί πως με την αφαίρεση των stopwords  παρατηρήθηκε ελάχιστη μείωση στα ποσοστά. Αυτό το φαινόμενο μπορεί να εξηγηθεί, καθώς σε μερικές περιπτώσεις η μείωση/τροποποίηση των λέξεων μπορεί να οδηγήσει στην αλλαγή νοήματος των προτάσεων και να υπάρξουν παρερμηνείες. Επίσης , υπάρχουν αναλυτικά όλα τα ποσοστά τόσο για τα train data όσο και για τα test data για όλους τους αλγορίθμους. 


4) Για τον σύνθετο πίνακα tfidf-pos τα καλύτερα ποσοστά παρατηρήθηκαν με τον svm. Τελικά κάνοντας το κατάλληλο parameter tunning τα ποσοστά στον svm και στον random forrest αυξήθηκαν σημαντικά. Χαρακτηριστικά το  accuracy του svc πήγε απο 66.93% σε 71.81% και για τον random forest από 62.68% σε 67.64%. 


5) Παρόλα αυτά , το καλύτερο ποσοστό από όλους τους αλγορίθμους ταξινόμησης για το test data ήταν με τον SGDClassifier για τον σύνθετο πίνακα στο 72.25%.


# Headers

In [ ]:
import pandas as pd
from pandas import read_csv
import numpy as np
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_predict
from sklearn.naive_bayes import GaussianNB
import re
import string
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# Reading files 

In [ ]:
train_set = pd.read_csv('data/train.csv')
test_set = pd.read_csv('data/impermium_verification_set.csv')
test_set_evaluation = pd.read_csv('data/impermium_verification_labels.csv')

#  Text Cleaning

In [ ]:
# dictionary inspired by https://stackoverflow.com/questions/43018030/replace-apostrophe-short-words-in-python
# Making words with apostrophes into mutiple words 
# More friendly for computer to understand the meaning of the phrase

contractions = {
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "I would",
"i'd've": "I would have",
"i'll": "I will",
"i'll've": "I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": " she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's":  "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

punctuation = '"'

row = 0
for comment in train_set.Comment:
    comment = comment.translate(str.maketrans('', '', punctuation))
    for word in comment.split():
        if word.lower() in contractions:
            comment = comment.replace(word, contractions[word.lower()])  
    train_set.loc[row, 'Comment'] = comment
    row += 1

row = 0
for comment in test_set.Comment:
    comment = comment.translate(str.maketrans('', '', punctuation)) 
    for word in comment.split():
        if word.lower() in contractions:
            comment = comment.replace(word, contractions[word.lower()]) 
    test_set.loc[row, 'Comment'] = comment
    row += 1        


In [ ]:
# List with indexes of rows of train_set we want to drop
rows_to_be_deleted_train = [] 
rows_to_be_deleted_test = [] 

# Lowercase
train_set.Comment = train_set.Comment.apply(lambda x: x.lower())
test_set.Comment = test_set.Comment.apply(lambda x: x.lower())

# Remove url
train_set.Comment = train_set.Comment.apply(lambda x: re.sub(r'http\S+', ' ', x))
test_set.Comment = test_set.Comment.apply(lambda x: re.sub(r'http\S+', ' ', x))


# Help us to seperate '\' from words 
train_set.Comment = train_set.Comment.apply(lambda x: x.replace('\\', ' \\'))
test_set.Comment = test_set.Comment.apply(lambda x: x.replace('\\', ' \\'))

# Remove '\xa0'
train_set.Comment = train_set.Comment.apply(lambda x: re.sub(r'\\xa0', ' ', x))
test_set.Comment = test_set.Comment.apply(lambda x: re.sub(r'\\xa0', ' ', x))

# Remove '\n'
train_set.Comment = train_set.Comment.apply(lambda x: re.sub(r'\\n', ' ', x))
test_set.Comment = test_set.Comment.apply(lambda x: re.sub(r'\\n', ' ', x))

# Remove '\u'
train_set.Comment = train_set.Comment.apply(lambda x: re.sub(r'\\u', ' ', x))
test_set.Comment = test_set.Comment.apply(lambda x: re.sub(r'\\u', ' ', x))

# Remove numbers
train_set.Comment = train_set.Comment.apply(lambda x:re.sub("\d+", " ", x))
train_set.Comment = train_set.Comment.apply(lambda x:re.sub("\d+", " ", x))

# Remove words that have '\'
train_set.Comment = train_set.Comment.apply(lambda x: re.sub(r'\\[a-z]*', ' ', x))
test_set.Comment = test_set.Comment.apply(lambda x: re.sub(r'\\[a-z]*', ' ', x))


# Remove punctuation
train_set.Comment = train_set.Comment.apply(lambda x: x.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))))
test_set.Comment = test_set.Comment.apply(lambda x: x.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))))

# Delete multiple consecutive spaces
train_set.Comment = train_set.Comment.apply(lambda x: ' '.join(x.split()))
test_set.Comment = test_set.Comment.apply(lambda x: ' '.join(x.split()))

# After text cleaning drop rows for test and train data that have "invalid" comment 

row = 0
for comment in train_set.Comment:
    if ((comment == "")==True or (comment == " ")==True): 
        rows_to_be_deleted_train.append(row)  
    row = row + 1

row = 0
for comment in test_set.Comment:
    if ((comment == "")==True or (comment == " ")==True): 
        rows_to_be_deleted_test.append(row)  
    row = row + 1    

train_set.drop(train_set.index[rows_to_be_deleted_train], inplace=True)  
test_set.drop(test_set.index[rows_to_be_deleted_test], inplace=True) 
test_set_evaluation.drop(test_set_evaluation.index[rows_to_be_deleted_test], inplace=True) 

In [ ]:
stopwords = list(STOPWORDS) 

# Creating bag of words for train and test set
bow_vectorizer = CountVectorizer()
bow_train = bow_vectorizer.fit_transform(train_set["Comment"])
bow_test =  bow_vectorizer.transform(test_set["Comment"])

# Actual insults of train data
train_y = train_set["Insult"].to_numpy()
train_y = train_y.astype('int')

# Actual test_set insults values
actual_test_set_y = test_set_evaluation["Insult"].to_numpy()
actual_test_set_y = actual_test_set_y.astype('int')

In [ ]:
# Trains data with model and evaluates classification 

def Find_Classification(model , my_array , train_y , my_model):

    #lists for k fold cross val
    list_accuracy=[]
    list_f1=[]

    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=3)

    #kf.split gives us 10 different splits 
    for train_index, test_index in kf.split(bow_train.toarray() , train_y):
        X_train, X_test = my_array[train_index], my_array[test_index]
        y_train, y_test = train_y[train_index], train_y[test_index]
        
        #fit and predict
        model.fit(X_train,y_train)
        y_pred= model.predict(X_test)
        
        #Save metrics to calculate average scores
        list_accuracy.append(100* metrics.accuracy_score(y_test,y_pred) )
        list_f1.append(100 *f1_score(y_test,y_pred) )

        
    print(my_model, "Accuracy metric", sum(list_accuracy) /len(list_accuracy))
    print(my_model, "F1 metric",sum(list_f1) /len(list_f1))
    print("\n")


# Improving scores on train set

## 1) Lemmatization

In [ ]:
train_set_comment_lem = train_set.Comment

import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
train_set_comment_lem = train_set_comment_lem.apply(lambda x: ' '.join([lemmatizer.lemmatize(w) for w in x.rstrip().split()]))
bow_train_with_lem = bow_vectorizer.fit_transform(train_set_comment_lem)

Find_Classification(GaussianNB(),  bow_train.toarray() , train_y, "NaiveBayes BOW")
Find_Classification(GaussianNB(),  bow_train_with_lem.toarray() , train_y, "NaiveBayes BOW with lemmatization")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
NaiveBayes BOW Accuracy metric 66.69215446893273
NaiveBayes BOW F1 metric 44.46627366590176


NaiveBayes BOW with lemmatization Accuracy metric 65.93201824840969
NaiveBayes BOW with lemmatization F1 metric 44.89016439185612




## 2) Stopwords

In [ ]:
bow_vectorizer_with_stopwords = CountVectorizer(stop_words=stopwords)
bow_train_with_stopwords = bow_vectorizer_with_stopwords.fit_transform(train_set["Comment"])
Find_Classification(GaussianNB(),  bow_train.toarray() , train_y, "NaiveBayes BOW")
Find_Classification(GaussianNB(),  bow_train_with_stopwords.toarray() , train_y, "NaiveBayes BOW with stopwords")

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'let', 'll', 'mustn', 're', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


NaiveBayes BOW Accuracy metric 66.69215446893273
NaiveBayes BOW F1 metric 44.46627366590176


NaiveBayes BOW with stopwords Accuracy metric 66.69221872389643
NaiveBayes BOW with stopwords F1 metric 44.55750026458257




## 3) Bigrams

In [ ]:
bow_vectorizer_with_bigrams = CountVectorizer(ngram_range=(2,2))
bow_train_with_bigrams = bow_vectorizer_with_bigrams.fit_transform(train_set["Comment"])
Find_Classification(GaussianNB(),  bow_train.toarray() , train_y, "NaiveBayes BOW")
Find_Classification(GaussianNB(),  bow_train_with_bigrams.toarray() , train_y, "NaiveBayes BOW with bigrams")

NaiveBayes BOW Accuracy metric 66.69215446893273
NaiveBayes BOW F1 metric 44.46627366590176


NaiveBayes BOW with bigrams Accuracy metric 68.94988112831717
NaiveBayes BOW with bigrams F1 metric 44.20455153394136




## 4) Laplace Smoothing

In [ ]:
Find_Classification(GaussianNB(),  bow_train.toarray() , train_y, "NaiveBayes BOW")
Find_Classification(MultinomialNB(alpha=1.0),  bow_train.toarray() , train_y, "Multinomial NaiveBayes BOW")

NaiveBayes BOW Accuracy metric 66.69215446893273
NaiveBayes BOW F1 metric 44.46627366590176


Multinomial NaiveBayes BOW Accuracy metric 79.13840519180107
Multinomial NaiveBayes BOW F1 metric 63.383834060714136




# Initial accuracy of NB model on test set

In [ ]:
model = GaussianNB()
model.fit(bow_train.toarray(),train_y)
y_pred = model.predict(bow_test.toarray())

print( "accuracy ",100 * metrics.accuracy_score(actual_test_set_y,y_pred))
print( "f1_Score ",100 * f1_score(actual_test_set_y,y_pred))

accuracy  51.856823266219244
f1_Score  51.487826871055


# Evaluating improvements on test set

## 1) Lemmatization

In [ ]:
train_set_comment_lem = train_set.Comment
test_set_comment_lem = test_set.Comment

# nltk.download('wordnet')
import nltk
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
train_set_comment_lem = train_set_comment_lem.apply(lambda x: ' '.join([lemmatizer.lemmatize(w) for w in x.rstrip().split()]))
bow_train_with_lem = bow_vectorizer.fit_transform(train_set_comment_lem)

test_set_comment_lem = test_set_comment_lem.apply(lambda x: ' '.join([lemmatizer.lemmatize(w) for w in x.rstrip().split()]))
bow_test_with_lem = bow_vectorizer.transform(test_set_comment_lem)

model = GaussianNB()
model.fit(bow_train_with_lem.toarray(),train_y)
y_pred = model.predict(bow_test_with_lem.toarray())

print( "accuracy ",100 * metrics.accuracy_score(actual_test_set_y,y_pred))
print( "f1_Score ",100 * f1_score(actual_test_set_y,y_pred))


accuracy  51.90156599552572
f1_Score  52.32815964523281


## 2) Stopwords

In [ ]:
bow_vectorizer_with_stopwords = CountVectorizer(stop_words=stopwords)
bow_train_with_stopwords = bow_vectorizer_with_stopwords.fit_transform(train_set["Comment"])
bow_test_with_stopwords = bow_vectorizer_with_stopwords.transform(test_set["Comment"])

model = GaussianNB()
model.fit(bow_train_with_stopwords.toarray(), train_y)
y_pred = model.predict(bow_test_with_stopwords.toarray())

print( "accuracy ",100 * metrics.accuracy_score(actual_test_set_y,y_pred))
print( "f1_Score ",100 * f1_score(actual_test_set_y,y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'let', 'll', 'mustn', 're', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


accuracy  51.81208053691275
f1_Score  51.464623704371334


## 3) Bigrams

In [ ]:
bow_vectorizer_with_bigrams = CountVectorizer(ngram_range=(2,2))
bow_train_with_bigrams = bow_vectorizer_with_bigrams.fit_transform(train_set["Comment"])
bow_test_with_bigrams = bow_vectorizer_with_bigrams.transform(test_set["Comment"])

model = GaussianNB()
model.fit(bow_train_with_bigrams.toarray(), train_y)
y_pred = model.predict(bow_test_with_bigrams.toarray())

print( "accuracy ",100 * metrics.accuracy_score(actual_test_set_y,y_pred))
print( "f1_Score ",100 * f1_score(actual_test_set_y,y_pred))


accuracy  55.70469798657718
f1_Score  52.90199809705044


## 4) Laplace Smoothing

In [ ]:
model = MultinomialNB(alpha=1.0)
model.fit(bow_train.toarray(),train_y)
y_pred = model.predict(bow_test.toarray())

print( "accuracy ",100 * metrics.accuracy_score(actual_test_set_y,y_pred))
print( "f1_Score ",100 * f1_score(actual_test_set_y,y_pred))

accuracy  68.14317673378076
f1_Score  63.59918200408998


## Conclusion: The best improvement technique was laplace smoothing. Accuracy increased from 51,85 % to 68,14 % and f1_score also increased from 51,48 % to 63.59 %. However, removing stopwords lead to a insigificant reduce of  accuracies. The reason is that in some occasions these techniques can change completely the meaning of the sentence and cause unpredictable results. 

# TF-IDF and Part-of-Speech

In [ ]:
# Updating part of speech array of 4 columns

def update_pos_array(pos_array, pos_list, total_words, index): 
  
    # occurances of pos (Adverbs, Verbs, Adjectives, Nouns)
    pos_counters = [0,0,0,0]
  
    for item in pos_list:
        if "RB" in item[1]: 
            pos_counters[0] += 1
        elif "VB" in item[1]:
            pos_counters[1] += 1
        elif "JJ" in item[1]:
            pos_counters[2] += 1
        elif "NN" in item[1]: 
            pos_counters[3] += 1

    # creating list of frequencies 
    pos_fractions = [x / total_words for x in pos_counters]
    # updating array
    pos_array[index][:] = pos_fractions 
  
    

## Creating pos array of test and train set


In [ ]:

# Columns: fractionAdverbs,fractionVerbs,fractionAdjectives,fractionNouns
pos_array_train = np.zeros((train_set.shape[0],4), dtype=float) 

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

index = 0
for comment in train_set.Comment:
    text = nltk.word_tokenize(comment)
    total_words = len(text)
    pos_list = nltk.pos_tag(text)
    update_pos_array(pos_array_train, pos_list, total_words, index)
    index = index + 1

pos_array_test = np.zeros((test_set.shape[0],4), dtype=float) 

index = 0
for comment in test_set.Comment:
    text = nltk.word_tokenize(comment)
    total_words = len(text)
    pos_list = nltk.pos_tag(text)
    update_pos_array(pos_array_test, pos_list, total_words, index)
    index = index + 1


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


## Creating new tfidf-pos array for test and train set

In [ ]:
from scipy import sparse

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))

tfidf_train = tfidf_vectorizer.fit_transform(train_set["Comment"])
tfidf_train = tfidf_train.toarray()
tfidf_train = np.hstack((tfidf_train, pos_array_train))
tfidf_train_sparsed = sparse.csr_matrix(tfidf_train)

tfidf_test  = tfidf_vectorizer.transform(test_set["Comment"])
tfidf_test = tfidf_test.toarray()
tfidf_test = np.hstack((tfidf_test, pos_array_test))
tfidf_test_sparsed = sparse.csr_matrix(tfidf_test)

## Training new tfidf-pos array

In [ ]:
# Training data with 2 models using default parameters
# We will experiment with parameters later on beating the benchmark with our test data

Find_Classification(SVC(),  tfidf_train_sparsed , train_y, "SVC TF-IDF PoS")
Find_Classification(RandomForestClassifier(),  tfidf_train_sparsed , train_y, "RandomForrest TF-IDF PoS")

SVC TF-IDF PoS Accuracy metric 80.63406798175158
SVC TF-IDF PoS F1 metric 45.318343608245264


RandomForrest TF-IDF PoS Accuracy metric 78.53061749020112
RandomForrest TF-IDF PoS F1 metric 34.007941070585




## Evaluating models on test set

In [ ]:
# SVC

model = SVC()
model.fit(tfidf_train_sparsed,train_y)
y_pred = model.predict(tfidf_test_sparsed)

print( "SVC accuracy ",100 * metrics.accuracy_score(actual_test_set_y,y_pred))
print( "SVC f1_Score",100 * f1_score(actual_test_set_y,y_pred))

# RandomForest

model = RandomForestClassifier()
model.fit(tfidf_train_sparsed,train_y)
y_pred = model.predict(tfidf_test_sparsed)

print( "\n\nRandomForest accuracy ",100 * metrics.accuracy_score(actual_test_set_y,y_pred))
print( "RandomForest f1_Score ",100 * f1_score(actual_test_set_y,y_pred))

SVC accuracy  66.9351230425056
SVC f1_Score 54.46703635243377


RandomForest accuracy  61.61073825503356
RandomForest f1_Score  37.73584905660377


## Beating the benchmark

In [ ]:
# SVC

model = SVC(kernel='rbf', C = 10,  gamma=0.1)
model.fit(tfidf_train_sparsed,train_y)
y_pred = model.predict(tfidf_test_sparsed)

print( "SVC accuracy ",100 * metrics.accuracy_score(actual_test_set_y,y_pred))
print( "SVC f1_Score ",100 * f1_score(actual_test_set_y,y_pred))

# RandomForrest

model = RandomForestClassifier(n_estimators= 100, max_depth= None, max_features = 0.4, random_state= 11)
model.fit(tfidf_train_sparsed,train_y)
y_pred = model.predict(tfidf_test_sparsed)

print( "\n\nRandomForest accuracy ",100 * metrics.accuracy_score(actual_test_set_y,y_pred))
print( "RandomForest f1_Score ",100 * f1_score(actual_test_set_y,y_pred))

SVC accuracy  71.81208053691275
SVC f1_Score  67.82431052093973


RandomForest accuracy  68.7248322147651
RandomForest f1_Score  60.125499144324024


In [ ]:
from sklearn.linear_model import SGDClassifier

model = SGDClassifier(loss='modified_huber',shuffle=True, random_state=100)
model.fit(tfidf_train_sparsed,train_y)
y_pred = model.predict(tfidf_test_sparsed)

print( "SGDClassifier accuracy ",100 * metrics.accuracy_score(actual_test_set_y,y_pred))
print( "SGDClassifier f1_Score ",100 * f1_score(actual_test_set_y,y_pred))

SGDClassifier accuracy  72.25950782997764
SGDClassifier f1_Score  67.64091858037577


## Conclusion: With models SVC and RandomForrest, after some tuning on the parameters we managed to increase accuracy from 66.93% to 71.81% and from 62.68% to 67.64% respectively. However, the best score was with another model (SGDClassifier) on accuracy metric around 72.25%.
